## Step 1: Ensure that libraries are imported

In [1]:
!pip3 install tira>=0.0.141 ir-datasets python-terrier==0.10.0


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!rm -Rf ~/.tira/extracted_datasets/ir-lab-wise-2024/subsampled-ms-marco-ir-lab-20250105-test
!rm -Rf ~/.tira/.archived

In [3]:
# This command loads and starts PyTerrier so that it also works in TIRA.

from tira.third_party_integrations import ensure_pyterrier_is_loaded

ensure_pyterrier_is_loaded()

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.0 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
# PyTerrier must be imported after `ensure_pyterrier_is_loaded` is called.

from pyterrier import started, init

if not started():
    init()

## Step 2: Load the dataset

In [5]:
from pyterrier import get_dataset

dataset = get_dataset('irds:ir-lab-wise-2024/subsampled-ms-marco-ir-lab-20250105-test')
dataset

IRDSDataset('ir-lab-wise-2024/subsampled-ms-marco-ir-lab-20250105-test')

## Step 3: Create the retrieval pipeline with TIRA

In this example, we will just use two existing retrieval components from TIREx: BM25 and DirichletLM, two lexical rankers.
We load the approaches via the TIRA API.

In [6]:
from tira.rest_api_client import Client

tira_client = Client()

In [7]:
llm = tira_client.pt.from_retriever_submission(
    approach='ir-lab-wise-2024/ir-wise-24-th25/BM25 + LLM Data Cleaning',
    dataset='subsampled-ms-marco-ir-lab-20250105-test',
)
llm

TiraSourceTransformer()

In [26]:
llm = tira_client.pt.from_retriever_submission(
    approach='ir-lab-wise-2024/ir-wise-24-th25/Baseline Readability Sentence',
    dataset='subsampled-ms-marco-deep-learning-20241201-training',
)
llm

Download: 1.09MiB [00:00, 9.88MiB/s]


Download finished. Extract...
Extraction finished:  /home/codespace/.tira/extracted_runs/ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training/ir-wise-24-th25


TiraSourceTransformer()

In [8]:
bm25 = tira_client.pt.from_retriever_submission(
    approach='ir-lab-wise-2024/ir-wise-24-uk-ir-1/BM25',
    dataset='subsampled-ms-marco-ir-lab-20250105-test',
)
bm25

TiraSourceTransformer()

In [32]:
bm25 = tira_client.pt.from_retriever_submission(
    approach='ir-lab-wise-2024/ir-wise-24-th25/Unchanged BM25 Baseline',
    dataset='subsampled-ms-marco-deep-learning-20241201-training',
)
bm25

Download: 1.11MiB [00:00, 10.1MiB/s]


Download finished. Extract...
Extraction finished:  /home/codespace/.tira/extracted_runs/ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training/ir-wise-24-th25


TiraSourceTransformer()

## Step 4: Measure effectiveness

Now let us measure the nDCG@10 effectiveness of both systems on the Touché 2020 task 1 dataset.

In [9]:
from pyterrier.pipelines import Experiment

experiment = Experiment(
    retr_systems=[
        llm,
        bm25,
    ],
    topics=dataset.get_topics("query"),
    qrels=dataset.get_qrels(),
    eval_metrics=["ndcg_cut_10"],
    names=[
        "CleaningLLM",
        "BM25",
    ],
    perquery=True,
)
experiment.sample(n=10)

Download from Zenodo: https://zenodo.org/records/14743268/files/subsampled-ms-marco-ir-lab-20250105-test-truths.zip


Download: 100%|██████████| 50.6k/50.6k [00:00<00:00, 59.3MiB/s]


Download finished. Extract...
Extraction finished:  /home/codespace/.tira/extracted_datasets/ir-lab-wise-2024/subsampled-ms-marco-ir-lab-20250105-test/


,name,qid,measure,value
57,BM25,45,ndcg_cut_10,1.000000
71,BM25,28,ndcg_cut_10,0.463926
62,BM25,36,ndcg_cut_10,0.220092
87,BM25,31,ndcg_cut_10,0.000000
70,BM25,4,ndcg_cut_10,1.000000
69,BM25,25,ndcg_cut_10,0.000000
79,BM25,29,ndcg_cut_10,1.000000
37,CleaningLLM,14,ndcg_cut_10,0.505360
24,CleaningLLM,4,ndcg_cut_10,0.861138
22,CleaningLLM,19,ndcg_cut_10,0.078398


In [10]:
experiment_bm25 = experiment[experiment["name"] == "BM25"]\
    .drop(columns=["name"])
experiment_llm = experiment[experiment["name"] == "CleaningLLM"]\
    .drop(columns=["name"])

experiment_paired = experiment_bm25.merge(
    experiment_llm,
    on=["qid", "measure"],
    suffixes=("_bm25", "_llm"),
)
experiment_paired.head(n=10)

,qid,measure,value_bm25,value_llm
0,10,ndcg_cut_10,0.546257,0.312529
1,11,ndcg_cut_10,0.063621,0.218195
2,12,ndcg_cut_10,0.224663,0.066254
3,13,ndcg_cut_10,1.000000,0.305235
4,14,ndcg_cut_10,0.921602,0.505360
5,16,ndcg_cut_10,0.643404,0.532141
6,17,ndcg_cut_10,0.000000,0.000000
7,18,ndcg_cut_10,0.000000,0.000000
8,19,ndcg_cut_10,0.425208,0.078398
9,2,ndcg_cut_10,1.000000,1.000000


## Step 5: Conduct hypothesis tests

On this _paired_ measurement data, we can now conduct _paired_ t-tests to test for statistical significance of given hypotheses.
Remember that the choice of your test depends (amongst other factors) on how the hypothesis is formulated.

Let us test some hypotheses to get a feeling of what this means:

#### Hypothesis 1: Der Einsatz von Large Language Models (LLMs) für das Data Cleaning zusätzlich zur BM25-Methode führt zu signifikant verbesserten NDCG@10-Werten im Vergleich zur BM25-Methode, da LLMs in der Lage sind, Datenkontexte und semantische Beziehungen besser zu erfassen.

Significance test: one-sided paired t-test \
Significance level: $\alpha = 0.05$ (or $p < 0.05$)

In [11]:
from scipy.stats import ttest_rel

ttest_rel(
    experiment_paired["value_llm"],
    experiment_paired["value_bm25"],
    alternative='two-sided',
).pvalue

9.043442907048412e-06

This is lower than the significance level. It suggests there is a statistically significant difference.

In [12]:
from scipy.stats import ttest_rel

ttest_rel(
    experiment_paired["value_llm"],
    experiment_paired["value_bm25"],
    alternative='greater',
).pvalue

0.9999954782785465

The probability p of the null hypothesis is higher than our significance level alpha.
So we cannot reject the null hypothesis and fail to confirm hypothesis 1.